物体検出

1. Google Driveに接続
1. データの取得
1. 学習のためのコードを取得
1. 学習

#### 1.Google Driveに接続

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 2.データの取得

Dataset: https://www.kaggle.com/datasets/sebastianpalaciob/plantvillage-for-object-detection-yolo

In [2]:
import glob
import shutil
from sklearn.model_selection import train_test_split
import os

data_path = "/content/drive/MyDrive/detection/apple/"
x_list = glob.glob(data_path + "*.jpg") + glob.glob(data_path + "*.JPG")
y_list = glob.glob(data_path + "*.txt")
x_train, x_test, y_train, y_test = train_test_split(x_list, y_list)
print("学習数", len(x_train))
print("テスト数", len(x_test))

学習数 150
テスト数 50


In [3]:
from tqdm import tqdm
import shutil
if os.path.exists("/content/train"):
  shutil.rmtree("/content/train")
if os.path.exists("/content/val"):
  shutil.rmtree("/content/val")
if not os.path.exists("/content/train"):
  os.mkdir("/content/train")
if not os.path.exists("/content/val"):
  os.mkdir("/content/val")
for f in tqdm(x_train):
  file_name = os.path.basename(f)
  shutil.copyfile(f, f"/content/train/{file_name}")
for f in tqdm(y_train):
  file_name = os.path.basename(f)
  shutil.copyfile(f, f"/content/train/{file_name}")
for f in tqdm(x_test):
  file_name = os.path.basename(f)
  shutil.copyfile(f, f"/content/val/{file_name}")
for f in tqdm(y_test):
  file_name = os.path.basename(f)
  shutil.copyfile(f, f"/content/val/{file_name}")

100%|██████████| 50/50 [00:00<00:00, 273.01it/s]


In [4]:
yaml = """
# here you need to specify the files train, test and validation txt
train: /content/train
val: /content/val

# number of classes in your dataset
nc: 1

# class names
names: ['apple_scab']
"""
with open("data.yaml", "w") as f:
  f.write(yaml)

#### 3.学習のためのコードを取得

In [5]:
!pip install jedi>=0.10 # 202302 追加

In [6]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16836, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16836 (delta 1), reused 6 (delta 0), pack-reused 16825
Receiving objects: 100% (16836/16836), 15.57 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (11550/11550), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


#### 4.学習

In [7]:
import torch
if torch.cuda.is_available():
  print("GPUの設定ができています")
else:
  print("GPUの設定ができていません。GPUを設定してください")

GPUの設定ができています


####ハイパーパラメーター

画像サイズは検出したいものが鉄筋断面と小さいので、大きめの1280にしています。  
エポック数は、数十から100にします。

####初回の学習は、次のように実行してください。

* model_type = 'yolov5m' #@param ["yolov5s", "yolov5m", "yolov5l"]
* #weights_dir = '' # 1から学習する場合
* #weights_dir = project_dir + "exp/weights/best.pt" # 自己学習した重みを使う場合

weights_dir = f'{model_type}.pt' # 80種類の分類で学習ずみの重みを使う場合
* 自己学習した重みを使う場合、は過去に学習した際に生成したbest.ptを使用する際に使用します。

In [8]:
project_dir = "/content/drive/MyDrive/detection/exps/"
image_size = 1280
batch_size = 8
epoch_num = 40
model_type = 'yolov5m' #@param ["yolov5s", "yolov5m", "yolov5l"]
# weights_dir = '' # 1から学習する場合
# weights_dir = project_dir + "exp/weights/best.pt" # 自己学習した重みを使う場合
weights_dir = f'{model_type}.pt' # 80種類の分類で学習ずみの重みを使う場合
hyp_file = 'hyp.scratch-low.yaml' #@param ["hyp.VOC.yaml", "hyp.scratch-high.yaml", "hyp.scratch-med.yaml", "hyp.scratch-low.yaml"]
hyp_file = f'./data/hyps/{hyp_file}'
arguments = f"--img {image_size} --batch {batch_size} --epochs {epoch_num} \
              --data /content/data.yaml --cfg ./models/{model_type}.yaml --weights '{weights_dir}' \
              --device 0 --project {project_dir} --hyp {hyp_file}"

In [9]:
!python train.py {arguments}

2024-08-10 04:14:16.986290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 04:14:17.245737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 04:14:17.324893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=./models/yolov5m.yaml, data=/content/data.yaml, hyp=./data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_sca